In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
caminho = "files/comunicando.pdf"
loader = PyPDFLoader(caminho)
paginas = loader.load()
len(paginas)

3

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=5,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)
len(documents)

10

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [13]:
from langchain_community.vectorstores import Chroma

diretorio = "files/chroma_vectorstore"

vector_store = Chroma.from_documents(
    documents=documents,
    embedding = embeddings_model,
    persist_directory=diretorio
)

In [14]:
vector_store._collection.count()

10

In [16]:
vector_store = Chroma(
    embedding_function=embeddings_model,
    persist_directory=diretorio
)

/tmp/ipykernel_65726/3326131476.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [17]:
pergunta = "Como instalar o Ollama?"

docs = vector_store.similarity_search(pergunta, k=5)
len(docs)

5

In [18]:
for doc in docs:
    print(doc.page_content)
    print(f"======{doc.metadata}")

singularity exec ollama.sif ollama run <modelo_llm> 
Agora sim, temos o modelo pronto na máquina remota para realizar requisições. 
 
 
4. Como Requisitar na sua Máquina? 
Da sua máquina, você vai realizar da mesma maneira que foi citado anteriormente: 
singularity exec ollama.sif ollama run <modelo_llm> 
Dessa maneira instalamos o modelo no container através de sua máquina, para aí sim você pode
======{'author': 'Lucas Abner', 'creationdate': '2025-07-10T11:05:46-07:00', 'creator': 'Microsoft Word', 'moddate': '2025-07-10T11:05:46-07:00', 'page': 1, 'page_label': '2', 'producer': 'PyPDF', 'source': 'files/comunicando.pdf', 'total_pages': 3}
Agora que temos o container em nossa máquina remota, e já fizemos o túnel necessario para escutar 
a requisição ollama, é preciso ativar esse container para conseguir usar a sua LLM no projeto, certo? 
Então vamos lá: 
singularity run –-nv ollama.sif & 
Viu? De maneira simples já colocamos nosso container para funcionar. Esse // & // comercial é pa